# Lecture Note 5: Synchronization

## Example: Multiple threads show their names on the monitor

In [ ]:
import threading
from time import sleep

def TaskAdd(A):
  global Num
  for _ in range(2):
    print(A+'\n')
    sleep(1)

if __name__ == "__main__":
  Num = 0
  T1 = threading.Thread(target=TaskAdd, args=('Thread T1',))
  T2 = threading.Thread(target=TaskAdd, args=('Thread T2',))
  T1.start()
  T2.start()
  T1.join()
  T2.join()


## Example: Multiple processes cooperatively increase a shared variable

In [ ]:
import multiprocessing

def TaskAdd(A):
	for _ in range(A):
		Num.value = Num.value + 1

if __name__ == "__main__":
  Num = multiprocessing.Value('i', 0)
  P1 = multiprocessing.Process(target=TaskAdd, args=(500000,))
  P2 = multiprocessing.Process(target=TaskAdd, args=(500000,))
  P1.start()
  P2.start()
  P1.join()
  P2.join()
  print('Total = %d' %Num.value)


# Section 3: Mutex Lock

## Example: Spin Lock

In [ ]:
import multiprocessing

def TaskP1():
  print('P1 is executing.\n')
  while (Num.value < 200000):
    pass
  print('P1 prints Num = %d\n' %Num.value)

def TaskP2():
  print('P2 is executing while P1 is in a spin lock.\n')
  for _ in range(500000):
    Num.value = Num.value + 1
  print('P2 prints Num = %d\n' %Num.value)

if __name__ == "__main__":
  Num = multiprocessing.Value('i', 0)
  P1 = multiprocessing.Process(target=TaskP1)
  P2 = multiprocessing.Process(target=TaskP2)
  P1.start()
  P2.start()
  P1.join()
  P2.join()


## Example: Using Python mutex locks

In [ ]:
import multiprocessing

def TaskAdd(A, lock):
  for _ in range(A):
    lock.acquire()
    Num.value = Num.value + 1
    lock.release()

if __name__ == "__main__":
  Num = multiprocessing.Value('i', 0)
  lock = multiprocessing.Lock()
  P1 = multiprocessing.Process(target=TaskAdd, args=(500000, lock, ))
  P2 = multiprocessing.Process(target=TaskAdd, args=(500000, lock, ))
  P1.start()
  P2.start()
  P1.join()
  P2.join()
  print('Total = %d' %Num.value)


# Section 4: Semaphores

## Example: Using Python semaphores

In [ ]:
import multiprocessing

def TaskAdd(A, semlock):
  for _ in range(A):
    semlock.acquire()
    Num.value = Num.value + 1
    semlock.release()

if __name__ == "__main__":
  Num = multiprocessing.Value('i', 0)
  semlock = multiprocessing.Semaphore(1)
  P1 = multiprocessing.Process(target=TaskAdd, args=(500000, semlock, ))
  P2 = multiprocessing.Process(target=TaskAdd, args=(500000, semlock, ))
  P1.start()
  P2.start()
  P1.join()
  P2.join()
  print('Total = %d' %Num.value)


# Section 5: Problems of Using Synchronization

## Example: Deadlock due to circular wait

In [ ]:
from time import sleep
import multiprocessing

def ProcTask(ProcName, lock1, lock2):
  print(ProcName + ' would like to enter the critical section.\n')
  lock1.acquire()
  sleep(1)
  print(ProcName + ' owns his first lock and waiting for the second lock ...\n' )
  lock2.acquire()
  print(ProcName + ' is in the critical section.\n' )
  lock2.release()
  lock1.release()

if __name__ == '__main__':
  lockA = multiprocessing.Lock()
  lockB = multiprocessing.Lock()

  P1 = multiprocessing.Process(target=ProcTask, args=('P1', lockA, lockB, ))
  P2 = multiprocessing.Process(target=ProcTask, args=('P2', lockB, lockA, ))
  P1.start()
  P2.start()
  P1.join()
  P2.join()


# End